In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import skimage.io
import albumentations as A

import random
import sys
import os

sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

from efficientnet_pytorch import EfficientNet
from tqdm.notebook import tqdm
import numpy as np

In [2]:
class Config:
    
    width=224
    height=224
    dim = (width, height)
    
    batch_size = 1
    n_classes = 6
    seed = 42
    
    folds = 5
    model_name = 'efficientnet-b0'
    pretrained_model = ['../input/checkpoint-fold11/checkpoint_Fold1.pt', '../input/checkpoint-fold222/checkpoint_Fold2.pt',
                        '../input/checkpoint-fold33333/checkpoint_Fold3.pt', '../input/checkpoint-fold444/checkpoint_Fold4.pt',
                        '../input/checkpoint-fold5555/checkpoint_Fold5.pt']

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(Config.seed)

In [3]:
path = '../input/prostate-cancer-grade-assessment'
train_file = 'train_images'
test_file = 'test_images'

In [4]:
N = 10
if os.path.exists('../input/prostate-cancer-grade-assessment/test_images'):
    subm = pd.read_csv(os.path.join(path, 'sample_submission.csv'))[['image_id']]
    image_folder = os.path.join(path, test_file)
else:
    subm = pd.read_csv(os.path.join(path, 'train.csv')).loc[:N ,['image_id']]
    image_folder = os.path.join(path, train_file)

In [5]:
def PlantModel(model_name=Config.model_name):
    model = EfficientNet.from_name(Config.model_name)
    fc = model._fc.in_features
    model._fc = nn.Sequential(nn.Linear(fc, 1000, bias=True),
                              nn.ReLU(),
                              nn.Dropout(p=.5),
                              nn.Linear(1000, Config.n_classes, bias=True))
    
    return model

In [6]:
tile_size = image_size = 256
n_tiles = 16

def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img) < n_tiles:
            img3 = np.pad(img3,[[0,N-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


class PlantData(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 sub_imgs=False,
                 transform=None
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.sub_imgs = sub_imgs
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))
        idxes = np.asarray(idxes) + self.n_tiles if self.sub_imgs else idxes

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                if self.transform is not None:
                    this_img = self.transform(image=this_img)['image']
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img
        
        if self.transform is not None:
            images = self.transform(image=images)['image']
        images = 255 - images
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        return torch.tensor(images)

In [7]:
transforms_test = A.Compose([
    A.Transpose(p=0.7),
    A.VerticalFlip(p=0.7),
    A.HorizontalFlip(p=0.7),
])

dataset_test = PlantData(subm, image_size , n_tiles, 0, transform=transforms_test)
test_dataloader = DataLoader(dataset_test, batch_size=Config.batch_size, shuffle=False, num_workers=4)

In [8]:
submission_values = {'Fold_1':[], 'Fold_2':[], 'Fold_3':[], 'Fold_4':[], 'Fold_5':[]}
tta_count = 10

for fold in range(Config.folds):
    print('#'*60)
    print('#'*60)
    print('\t\t\tFOLD {}/{}'.format(fold+1, Config.folds))
    print('#'*60)
    print('#'*60)

    test_end = 0

    model = PlantModel()
    model.load_state_dict(torch.load(Config.pretrained_model[fold]))
    model.to(Config.device)

    model.eval()
    for tta in range(tta_count):
        test_preds = None
        for images in tqdm(test_dataloader):
            images = images.to(Config.device)
            outputs = model(images)

            with torch.no_grad():
                if test_preds is None:
                    test_preds = outputs.detach().cpu()
                else:
                    test_preds = torch.cat((test_preds, outputs.detach().cpu()), dim=0)
        test_end += test_preds
    submission_values[f'Fold_{fold+1}'] = test_end / tta_count
        
subm['isup_grade'] = (submission_values['Fold_1'].argmax(1) + submission_values['Fold_2'].argmax(1) + \
                     submission_values['Fold_3'].argmax(1) + submission_values['Fold_4'].argmax(1) + \
                     submission_values['Fold_5'].argmax(1)) / Config.folds

subm[['image_id','isup_grade']].to_csv('submission.csv', index=False)

############################################################
############################################################
			FOLD 1/5
############################################################
############################################################



############################################################
############################################################
			FOLD 2/5
############################################################
############################################################



############################################################
############################################################
			FOLD 3/5
############################################################
############################################################



############################################################
############################################################
			FOLD 4/5
############################################################
############################################################



############################################################
############################################################
			FOLD 5/5
############################################################
############################################################


/opt/conda/conda-bld/pytorch_1587428398394/work/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


In [10]:
submission_values

{'Fold_1': tensor([[ 1.2855, -1.9779, -3.6231, -3.2041, -5.2469, -4.5954],
         [ 0.4218, -1.6543, -2.7362, -1.6827, -3.8724, -3.2097],
         [ 0.5008, -1.3983, -4.5042, -1.9908, -4.6689, -3.0942],
         [ 0.0624, -1.5094, -3.6652, -1.3191, -4.1768, -3.5422],
         [ 0.7114, -1.8268, -3.6209, -2.1127, -4.8308, -3.8259],
         [ 0.6669, -1.7786, -3.6467, -2.1740, -4.4463, -3.5304],
         [ 1.5694, -2.2359, -3.5185, -2.9247, -4.8155, -4.1516],
         [-0.7534, -0.8090, -2.8549, -2.1364, -3.9857, -3.9305],
         [-1.1240,  0.1009, -2.4252, -1.8129, -3.1706, -3.4681],
         [ 2.3041, -2.6978, -5.0393, -4.9152, -7.5870, -6.8115],
         [ 1.5986, -2.3075, -4.3244, -2.9565, -5.4801, -4.5381]]),
 'Fold_2': tensor([[ 0.9262, -1.4797, -8.8420, -4.3275, -7.2276, -9.6192],
         [ 0.5878, -1.2255, -8.8452, -4.4356, -7.3614, -9.9734],
         [ 0.5363, -1.4714, -8.3590, -3.2098, -7.3397, -9.0109],
         [ 0.3406, -1.2509, -7.8972, -3.4035, -6.9816, -9.0021],
   

In [9]:
subm

,image_id,isup_grade
0,0005f7aaab2800f6170c399693a96917,0
1,000920ad0b612851f8e01bcc880d9b3d,0
2,0018ae58b01bdadc8e347995b69f99aa,0
3,001c62abd11fa4b57bf7a6c603a11bb9,0
4,001d865e65ef5d2579c190a0e0350d8f,0
5,002a4db09dad406c85505a00fb6f6144,0
6,003046e27c8ead3e3db155780dc5498e,0
7,0032bfa835ce0f43a92ae0bbab6871cb,1
8,003a91841da04a5a31f808fb5c21538a,0
9,003d4dd6bd61221ebc0bfb9350db333f,0
